In [1]:
import mlflow as mf
from pathlib import Path
import itertools
import pandas as pd
import numpy as np

In [2]:
mf.set_tracking_uri(str(Path('../code/train/mlruns').absolute()))

In [3]:
algs = ['lgb', 'svr','rf','mlp','ada','resnet1d','spectroresnet','mlpbp','unet1d','ppgiabp']
tars = ['SP','DP']
mets = ['mae', 'me', 'std', 'mase']
mets2 = [i+'_'+j for i, j in itertools.product(tars, mets)]
dbs = ['sensors', 'uci2', 'bcg', 'ppgbp']

In [4]:
l_df= []
for db in dbs:
    df_metrics = pd.DataFrame(index=algs,columns=mets2)
    for alg in algs:
        if not(alg in ['resnet1d','spectroresnet','mlpbp','unet1d','ppgiabp']):
            for tar in tars:
                print(db, alg, tar)
                exp = mf.get_experiment_by_name(f'mix-{db}-{alg}-{tar}')
                if not(exp is None):
                    runs = mf.search_runs([exp.experiment_id])
                    runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                    runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                    if runs.shape[0] > 0:
                        cols1 = [tar+'_'+m for m in mets if m!='mase']
                        cols2 = ['metrics.ts/'+m for m in cols1]
                        df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                        df_metrics.loc[alg,f'{tar}_mase'] = runs.iloc[0][f'metrics.ts/{tar}_mae']/runs.iloc[0][f'metrics.nv/{tar}_mae'] *100
        else: 
            print(db, alg)
            exp = mf.get_experiment_by_name(f'mix-{db}-{alg}')
            if not(exp is None):
                runs = mf.search_runs([exp.experiment_id])
                runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                if runs.shape[0] > 0:
                    cols1 = [i+'_'+j for i, j in itertools.product(tars, mets) if j != 'mase']
                    cols2 = ['metrics.test/'+m.replace('SP','sbp').replace('DP','dbp') for m in cols1]
                    
                    df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                    df_metrics.loc[alg,f'SP_mase'] = runs.iloc[0][f'metrics.test/sbp_mae']/runs.iloc[0][f'metrics.nv/sbp_mae'] *100
                    df_metrics.loc[alg,f'DP_mase'] = runs.iloc[0][f'metrics.test/dbp_mae']/runs.iloc[0][f'metrics.nv/dbp_mae'] *100
                
    l_df.append(df_metrics)
df_metrics = pd.concat(l_df)

sensors lgb SP
sensors lgb DP
sensors svr SP
sensors svr DP
sensors rf SP
sensors rf DP
sensors mlp SP
sensors mlp DP
sensors ada SP
sensors ada DP
sensors resnet1d
sensors spectroresnet
sensors mlpbp
sensors unet1d
sensors ppgiabp
uci2 lgb SP
uci2 lgb DP
uci2 svr SP
uci2 svr DP
uci2 rf SP
uci2 rf DP
uci2 mlp SP
uci2 mlp DP
uci2 ada SP
uci2 ada DP
uci2 resnet1d
uci2 spectroresnet
uci2 mlpbp
uci2 unet1d
uci2 ppgiabp
bcg lgb SP
bcg lgb DP
bcg svr SP
bcg svr DP
bcg rf SP
bcg rf DP
bcg mlp SP
bcg mlp DP
bcg ada SP
bcg ada DP
bcg resnet1d
bcg spectroresnet
bcg mlpbp
bcg unet1d
bcg ppgiabp
ppgbp lgb SP
ppgbp lgb DP
ppgbp svr SP
ppgbp svr DP
ppgbp rf SP
ppgbp rf DP
ppgbp mlp SP
ppgbp mlp DP
ppgbp ada SP
ppgbp ada DP
ppgbp resnet1d
ppgbp spectroresnet
ppgbp mlpbp
ppgbp unet1d
ppgbp ppgiabp


In [5]:
df_metrics

,SP_mae,SP_me,SP_std,SP_mase,DP_mae,DP_me,DP_std,DP_mase
lgb,14.786,-0.057,18.533,80.019483,7.666,0.008,9.929,82.759365
svr,15.109,0.009,18.882,81.767507,7.814,-1.443,10.212,84.35712
rf,15.274,-0.02,19.064,82.660461,7.878,0.014,10.147,85.048041
mlp,15.522,-1.754,19.414,84.002598,7.977,0.116,10.278,86.116809
ada,14.354,-0.12,18.125,77.681567,7.276,-0.27,9.576,78.549066
resnet1d,18.517,1.248,22.563,100.211062,9.045,-1.764,11.639,97.646551
spectroresnet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mlpbp,18.363,-0.14,22.493,99.377638,9.262,0.232,11.601,99.989204
unet1d,15.149,1.179,18.861,81.983981,7.702,0.58,9.924,83.148008
ppgiabp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
l_df= []
for db in ['sensors85']:
    df_metrics = pd.DataFrame(index=algs,columns=mets2)
    for alg in algs:
        if not(alg in ['resnet1d','spectroresnet','mlpbp','unet1d','ppgiabp']):
            for tar in tars:
                print(db, alg, tar)
                exp = mf.get_experiment_by_name(f'{db}-{alg}-{tar}')
                if not(exp is None):
                    runs = mf.search_runs([exp.experiment_id])
                    runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                    runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                    if runs.shape[0] > 0:
                        cols1 = [tar+'_'+m for m in mets if m!='mase']
                        cols2 = ['metrics.ts/'+m for m in cols1]
                        df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                        df_metrics.loc[alg,f'{tar}_mase'] = runs.iloc[0][f'metrics.ts/{tar}_mae']/runs.iloc[0][f'metrics.nv/{tar}_mae'] *100
        else: 
            print(db, alg)
            exp = mf.get_experiment_by_name(f'{db}-{alg}')
            if not(exp is None):
                runs = mf.search_runs([exp.experiment_id])
                runs = runs[runs['tags.mlflow.parentRunId'].isnull()].reset_index(drop=True)
                runs = runs[runs.status=='FINISHED'].reset_index(drop=True)
                if runs.shape[0] > 0:
                    cols1 = [i+'_'+j for i, j in itertools.product(tars, mets) if j != 'mase']
                    cols2 = ['metrics.test/'+m.replace('SP','sbp').replace('DP','dbp') for m in cols1]
                    
                    df_metrics.loc[alg,cols1] = runs.iloc[0][cols2].values
                    df_metrics.loc[alg,f'SP_mase'] = runs.iloc[0][f'metrics.test/sbp_mae']/runs.iloc[0][f'metrics.nv/sbp_mae'] *100
                    df_metrics.loc[alg,f'DP_mase'] = runs.iloc[0][f'metrics.test/dbp_mae']/runs.iloc[0][f'metrics.nv/dbp_mae'] *100
                
    l_df.append(df_metrics)
df_metrics = pd.concat(l_df)

sensors85 lgb SP
sensors85 lgb DP
sensors85 svr SP
sensors85 svr DP
sensors85 rf SP
sensors85 rf DP
sensors85 mlp SP
sensors85 mlp DP
sensors85 ada SP
sensors85 ada DP
sensors85 resnet1d
sensors85 spectroresnet
sensors85 mlpbp
sensors85 unet1d
sensors85 ppgiabp


In [7]:
df_metrics

,SP_mae,SP_me,SP_std,SP_mase,DP_mae,DP_me,DP_std,DP_mase
lgb,16.252,-0.465,20.181,87.4892,8.43,0.172,10.729,90.4701
svr,16.447,-0.277,20.511,88.539,8.163,-1.381,10.555,87.6046
rf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mlp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
resnet1d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
spectroresnet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mlpbp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unet1d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ppgiabp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
